In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import os

# URL da página principal com as empresas
BASE_URL = "https://statusinvest.com.br/ipo/acoes"

# Cabeçalhos para simular um navegador
headers = {
    "User-Agent": ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                   "AppleWebKit/537.36 (KHTML, like Gecko) "
                   "Chrome/91.0.4472.124 Safari/537.36")
}

# Diretório para salvar os PDFs
download_folder = "../data/minuta/"
os.makedirs(download_folder, exist_ok=True)

# Função para baixar o PDF
def download_pdf(pdf_url, filename):
    try:
        # Faz a requisição para o PDF
        pdf_response = requests.get(pdf_url, headers=headers)
        pdf_response.raise_for_status()  # Verifica se houve erro na requisição
        with open(filename, 'wb') as pdf_file:
            pdf_file.write(pdf_response.content)  # Salva o conteúdo como PDF
        print(f"  PDF baixado com sucesso: {filename}")
    except requests.exceptions.RequestException as e:
        print(f"  Erro ao baixar o PDF: {e}")

# 1. Obter a página principal e extrair os blocos de empresas com status "Protocolado" (data-status="0")
response = requests.get(BASE_URL, headers=headers)
if response.status_code != 200:
    print(f"Erro ao carregar a página principal: {response.status_code}")
    exit()

soup = BeautifulSoup(response.text, 'html.parser')

# Filtra os blocos das empresas com status "Protocolado"
company_divs = soup.find_all("div", attrs={"data-status": "0"})

# Extrai os links de cada empresa (usamos um conjunto para evitar duplicatas)
company_links = set()
for div in company_divs:
    a_tag = div.find("a", href=True)
    if a_tag:
        company_links.add(a_tag["href"])

print("Links das empresas com status Protocolado encontrados:")
for comp_link in company_links:
    # Se o link não for absoluto, junta com a URL base
    full_company_link = comp_link if comp_link.startswith("http") else urljoin(BASE_URL, comp_link)
    print(full_company_link)

print("\nAgora, vamos obter os links de PDF para cada empresa:")

# 2. Para cada empresa, acessar sua página e procurar o link para a página da CVM (que já é o link para baixar o PDF)
for comp_link in company_links:
    full_company_link = comp_link if comp_link.startswith("http") else urljoin(BASE_URL, comp_link)
    print(f"\nEmpresa: {full_company_link}")

    comp_response = requests.get(full_company_link, headers=headers)
    if comp_response.status_code != 200:
        print(f"  Erro ao acessar a página da empresa: {comp_response.status_code}")
        continue

    comp_soup = BeautifulSoup(comp_response.text, 'html.parser')
    
    # Procura por todos os links que contenham "sistemas.cvm.gov.br/dados/ofeanal"
    cvm_link_tags = comp_soup.find_all('a', href=lambda href: href and "sistemas.cvm.gov.br/dados/ofeanal" in href)
    
    if not cvm_link_tags:
        print("  Nenhum link da CVM encontrado nesta empresa.")
        continue

    # Para cada link da CVM encontrado, agora vamos tentar acessar diretamente o PDF
    for cvm_tag in cvm_link_tags:
        cvm_url = cvm_tag['href']
        print(f"  Página CVM: {cvm_url}")

        # Verifica se o link para o PDF é válido
        if cvm_url.lower().endswith('.pdf'):
            # Obtém o nome da empresa da URL e remove caracteres especiais
            company_name = full_company_link.split("/")[-1].replace("-", "_").replace(" ", "_")
            filename = os.path.join(download_folder, f"{company_name}_{cvm_url.split('/')[-1]}")
            download_pdf(cvm_url, filename)
        else:
            print("    O link não é um PDF direto.")
